## Terrain Correction Calculation

Outline:
 - Read in tiff of dem
 - Resample DEM to 59 x 59 m to reduce computation time
 - Create prisms model of DEM
 - Read in csv file of PPK positions
 - Sample the DEM at the measurment locations
 - Calculate the gravity anomaly from the model at the measurement locations, with the sampled elevation from the DEM
 - Find the difference between bouguer slab correction and model results with sampled elevation from DEM - this is the terrain correction

In [10]:
import numpy as np
from scipy.interpolate import  interp1d #RegularGridInterpolator,, RectBivariateSpline
import matplotlib.pyplot as plt
import rioxarray
# import fiona
# import rasterio.mask
import pandas as pd
# import verde as vd
# import geopandas as gpd
import time
# from shapely.geometry import *
# import rasterio as rs

import numpy as np
import xarray as xr
# import pyvista as pv
import harmonica as hm
from shapely import geometry

#from discretize.utils import mkvc


In [3]:
#read in the data measurement locations
profile4 = pd.read_csv('Data/prof4_meas.csv')
profile7a = pd.read_csv('Data/prof7a_meas.csv')
longa = pd.read_csv('Data/longa_meas.csv')

In [15]:
def open_raster(file): #create a function to read in the dem in tif format and convert it to an xarray which is used by harmonica
    #note function returns a dataArray and a Dataset which are different things - we use the dataArray - and also an array of x,y,z values
    geotiff_da = rioxarray.open_rasterio(file)
    geotiff_ds = geotiff_da.to_dataset('band') # Covert our xarray.DataArray into a xarray.Dataset
    geotiff_ds = geotiff_ds.rename({1: 'topo'}) # Rename the variable to a more useful name in the dataset

    geo_tiff_topo = geotiff_da[0] # in the dataArray select just the first variable which is the topography

    [x_topo, y_topo] = np.meshgrid(geo_tiff_topo.x.values, geo_tiff_topo.y.values) # the x and y values are read in as a single array for each and we want to create a grid of all values
    z_topo = geo_tiff_topo.values # defining the z component as the topography

    topo_xyz = np.c_[x_topo.ravel(), y_topo.ravel(), z_topo.ravel()] # we combine the x,y and z information into one array to be returned as a standard array

    topo_xr = xr.DataArray(geo_tiff_topo.values, # create the DataArray
    coords={'y': y_topo[:,0],'x': x_topo[0,:]}, 
    dims=["y", "x"])

    return topo_xr #returns the dataset, numpy array and dataArray

surf_inside = open_raster('Model_components/surf_inside.tif')
bed_outside = open_raster('Model_components/bed_outside.tif')
surf_xr_resamp = open_raster('Model_components/surf_xr_resamp.tif')
mask_geom_small = np.loadtxt('Model_components/mask_geom_small.csv', delimiter=',')
mask_rock_small = np.loadtxt('Model_components/mask_rock_small.csv', delimiter=',')
band_xr =  open_raster('Model_components/band_xr.tif')
dist_norm_xr = open_raster('Model_components/dist_norm_xr.tif')
mod_xr = open_raster('Model_components/mod_xr.tif')
center_points = pd.read_csv('Model_components/center_points_250m_nad83.csv')

In [12]:
# create simple shape function
dist_norm = np.arange(0,1.1,0.1)
#uncomment shape desired
y =  dist_norm**1/4 #V-shape
# y =  dist_norm**2/4 #U-shape
# y =  dist_norm**2.8/4 #wide-U-shape
parab_y = ((max(y) - y)/max(y)) #rearrange to make parabola to required function
parab_y = parab_y[::-1]
shape_func = interp1d(dist_norm, parab_y) #create shape function


In [17]:
#apply function within model area
thick_normal_xr = dist_norm_xr.copy()
for i in range(-1, 2):
    if i == 0:
        thick_normal_xr = thick_normal_xr.where(mod_xr != i, 0)
    if i == 1:
        thick_normal_xr = thick_normal_xr.where(mod_xr != i, shape_func(dist_norm_xr.where(mod_xr == i)))

x_dp = xr.DataArray(center_points['xcoord']) #turn the x and y into dataArrays so they can be used to sample the dem
y_dp = xr.DataArray(center_points['ycoord'])
center_points['thick_norm'] = thick_normal_xr.sel(x=x_dp, y=y_dp, method='nearest') #extract the thick norm values along the inversion points

# Outside Gravity Calculation

In [18]:
#calculate contribution from area outside model area
density = bed_outside.copy() # create a density data array
density.values[:] = 917 - 2700  # replace every value for the density of the topography
prisms = hm.prism_layer( # this is where we create the model of the topography, by creating a layer of prisms with varying elevation based on the dem
    (bed_outside.x.values, bed_outside.y.values),
    surface=bed_outside,
    reference=surf_xr_resamp,
    properties={"density": density},
)

# meas_grav['outer_grav'] = prisms.prism_layer.gravity((meas_grav['x_8N'], meas_grav['y_8N'], meas_grav['Elev']), field="g_z")

profile4['outer_grav'] = prisms.prism_layer.gravity((profile4['x_8N'], profile4['y_8N'], profile4['Elev']), field="g_z")
profile7a['outer_grav'] = prisms.prism_layer.gravity((profile7a['x_8N'], profile7a['y_8N'], profile7a['Elev']), field="g_z")
longa['outer_grav'] = prisms.prism_layer.gravity((longa['x_8N'], longa['y_8N'], longa['Elev']), field="g_z")

# Inversion

In [19]:
#set up points to be be inverted, starting model and density
param_start = np.ones_like(center_points['distance'])*1550

density_model = surf_inside.copy()
density_model.values[:] = 917 - 2700
meas_points = longa.copy()
g_meas_c10 = longa.boug_anom_2700

In [20]:
#inversion functions
#define inversion function - takes parameters which is bed elevation
def model(parameters0):
    params_xr = surf_inside.copy()
    param_norm = parameters0/center_points['thick_norm']
    for i in range(len(center_points)):
        dist_band = center_points['distance'].iloc[i]
        params_xr = params_xr.where(band_xr != dist_band, thick_normal_xr*param_norm[i])
    #final edit to make sure areas outside polygon are set to 0 thickness
    params_xr = params_xr.where(mask_geom_small == 1, 0)
    params_xr = params_xr.where(mask_rock_small == 0, 0)

    prisms_glacier = hm.prism_layer(
        (params_xr.x.values, params_xr.y.values),
        surface=surf_inside - params_xr,
        reference=surf_inside,
        properties={"density": density_model},
    )

    # g_sed_basin = prisms_glacier.prism_layer.gravity((meas_grav.x_8N, meas_grav.y_8N, meas_grav.Elev), field="g_z")
    g_inside = prisms_glacier.prism_layer.gravity((longa.x_8N, longa.y_8N, longa.Elev), field="g_z")
    g_inside_p4 = prisms_glacier.prism_layer.gravity((profile4['x_8N'], profile4['y_8N'], profile4['Elev']), field="g_z")
    g_tot = (g_inside - g_inside_p4[0]) + (longa['outer_grav'] - profile4['outer_grav'].iloc[0])

    return g_tot

def misfit(meas, model): #takes the measurements, the model, optionally start and end index of section
    e_1 = (2*np.sum(abs(meas - model)))/(np.sum(abs(meas - model)) +  np.sum(abs(meas + model))) #sum of differences
    return e_1


In [21]:
#calculate tolerance
Ugrs = longa['error'] 
test_meas = g_meas_c10
noise_misfit = []
for i in range(100):
    test_noise = test_meas + np.random.normal(0, Ugrs, len(test_meas))
    noise_misfit.append(misfit(test_meas, test_noise))

tolerance = np.mean(noise_misfit)

In [22]:
# set max and min limits
p_min = param_start -600  
p_max = param_start+400 

In [22]:
#vfsa

def T(kt, c, T0): #Temperature function - kt is iteration of temperature cooling, c is rate of cooling, T0 is starting temp (0-1)
    T = np.exp(-c*kt)*T0
    return T

model_runs = 1000 #max number of model runs
n_iterations = 1000#5000 # max number of iterations at each temperature
max_accepts = 500#1000 #max number of acceptances before moving on to next temperature
max_it_attempts = 200 #max number of times model will try to find updated model within max/min bounds before restarting iteration
reject_timeout = 60*15 #if this a model hasn't been accepted for 60 minutes exit model run
write_time_start = time.time() #start the timer for writing out the results
write_timeout = 30*1 #how often to write results - in seconds

M_start = model(param_start)
E_start = misfit(g_meas_c10, M_start)

inversion_misfits = [E_start] 
inversion_temp = [0]
inversion_results = M_start
inversion_parameters = param_start.copy()


mi_len = 3 #define number of closest points to include when assessing misfit over pertub area -- could also be a distance?
dist_perturb = 500 #distance within which to include points to perturb - relative to randomly chosen perturb points
dist_smooth = 750
param_weight_avg = np.array([4, 1, 1, 1])
param_weight_dists = np.array([0, 200, 400, 760 ])
weight_avg_fn = interp1d(param_weight_dists, param_weight_avg)

i_low = 8
i_up = -10

start_time = time.time()
tol_time = np.array([0, 0])

while tol_time[1] <= 100:

    M_old = np.copy(M_start) # define M_old and E_old - which will be updated as model run proceeds
    E_old = E_start
    params_old = param_start.copy() # define parameter dictionary for pertubation
    params_smoothed = param_start.copy() # define parameter dictionary for pertubation

    tol = 0 #set tolerance flag to 0
    t=0 #set temperature iteration to 0
    accept_time = time.time() #reset this with model run so clock starts over
 
    while time.time() < (accept_time + reject_timeout): #while a new model had been accepted in less than timeout
        # print ('t= ' + str(t)) #write the temperature
        temp_perturb = T(t, 0.25, 0.5) #temperature based on number of temp iterations and parameters
        temp_accept = T(t, 0.5, 0.1) #temperature based on number of temp iterations and parameters
        if tol == 1: #break out of model run if tolerance has been reached
            break
        i=0 #for start of each temperature reset i count to 0
        num_accept=0 #reset number of acceptances for each new temperature

        while i <= n_iterations and num_accept <= max_accepts and time.time() < (accept_time + reject_timeout): #continue to iterate at this temperature until either max iterations has been reached, max acceptances has been reached or no new models have been accepted under timeout

            perturb_points = dist_points.iloc[i_low:i_up].copy() #make a copy so not alterning dist points
            pi = np.random.choice(len(perturb_points)) #randomly select point to pertub it and the values around it
            meas_points['dist_pi'] = np.sqrt((perturb_points['xcoord'].iloc[pi] - meas_points['x_8N'])**2 + (perturb_points['ycoord'].iloc[pi] - meas_points['y_8N'])**2)
            meas_points = meas_points.sort_values(by='dist_pi')
            mp_i = meas_points.index[:mi_len] #take the 5 closest points -- could also change to taking closes points by distance
            E_old = misfit(g_meas_c10[mp_i], M_old[mp_i]) #calculate old error for this section
            if E_old >= tolerance:# and pi != p4_id:
                params_new = params_old.copy()
                perturb_points['dist_pi'] = np.sqrt((perturb_points['xcoord'].iloc[pi] - perturb_points['xcoord'])**2 + (perturb_points['ycoord'].iloc[pi] - perturb_points['ycoord'])**2)
                pi_i = perturb_points.index[perturb_points['dist_pi'] <= dist_perturb] #take the points within the perturb distance
                for p in pi_i:

                    it_attempt=0 #count for how many times value has tried to be updated
                    accept_param = 0
                    while it_attempt < max_it_attempts and accept_param==0:
                        ui = np.random.uniform(0,1) #select random number from uniform distribution
                        yi = np.sign(ui - 0.5) * temp_perturb*((1 + (1/temp_perturb))**(abs(2*ui - 1)) - 1) #dervie altertation term based on random number and temperature
                        point_pert = yi*(p_max[p] - p_min[p])
                        pi_new = params_new[p] + point_pert#get new value for parameter based on max/min allowed values and previous iteration
                        it_attempt += 1
                        if pi_new <= p_max[p] and pi_new >= p_min[p]:
                            params_new[p] = pi_new
                            accept_param = 1
                
                params_smoothed = np.copy(params_new) #copy to apply weighting function
                smooth_df = dist_points.iloc[i_low:i_up].copy() #make a copy so not alter dist df
                for p in pi_i:
                    # if p != p4_id:
                    smooth_df['dist_pi'] = np.sqrt((smooth_df['distance'].loc[p] - smooth_df['distance'])**2)
                    si = smooth_df.index[smooth_df['dist_pi'] <= dist_smooth]
                    weighting = weight_avg_fn(smooth_df.loc[si.values]['dist_pi'])
                    weighted_pnts = np.dot(params_new[si], weighting) #multiply selected points by their weighting
                    point_avg = np.sum(weighted_pnts)/np.sum(weighting) #divide sum of weighted points by weighting
                    params_smoothed[p] = point_avg
                params_smoothed[:i_low] = params_smoothed[i_low]
                params_smoothed[i_up:] = params_smoothed[i_up-1]

            accept = 0 #reset acceptance flag
            M_new = model(params_smoothed) #calulate anomaly with new parameter values
            E_new = misfit(g_meas_c10[mp_i], M_new[mp_i]) #calculate new error
            delta_E = E_new - E_old #check compared to old error
            if delta_E < 0: #if new error smaller accept new model
                accept = 1
                    
            if delta_E >= 0:
                P = np.exp(-delta_E/temp_accept) #otherwise calculate probability based on error difference and temp
                r = np.random.uniform(0,1)
                if P>r: #if probability greater than random number accept new model -- but don't update error as it is not new lowest
                    accept = 1

            if accept == 1:
                params_old = params_smoothed.copy() #update parameters to these new ones
                E_old = E_new.copy()
                M_old = np.copy(M_new) # model becomes old model
                accept_time = time.time() #reset time since last acceptance
                num_accept += 1 #add to number of acceptances at this temperature
                E_all = misfit(g_meas_c10, M_new) # calculate misfit over whole model - this will be used for tolerance
                
                if E_all < tolerance:
                    tol = 1
                    # print ('tolerance reached', t, i)
                    tol_time[0] = time.time() - start_time
                    tol_time[1] = tol_time[1] + 1

                    inversion_results = np.vstack((inversion_results, M_old))
                    inversion_misfits.append(E_all)
                    inversion_temp.append(t)
                    inversion_parameters = np.vstack((inversion_parameters, params_old))

                    np.savetxt('inversion_results/3Dmisfit.csv', inversion_misfits, delimiter=",")
                    np.savetxt('inversion_results/3Dtemp.csv', inversion_temp, delimiter=",")
                    np.savetxt('inversion_results/3Dresult.csv', inversion_results, delimiter=",")
                    np.savetxt('inversion_results/3Dparam.csv', inversion_parameters, delimiter=",")
                    np.savetxt('inversion_results/tol_time.csv', tol_time, delimiter=',')

                    break
            i+=1 #increase count for number of iterations

        t+=1
        if tol == 1:
            break

KeyboardInterrupt: 

In [24]:
def normal_dist(x):
    mean = np.mean(x)
    sd = np.std(x)
    # prob_density = (np.pi*sd) * np.exp(-0.5*((x-mean)/sd)**2)
    return mean, sd

In [25]:

inversion_filt = [i for i, x in enumerate(inversion_misfits) if x <= tolerance+0.02]
inversion_filt = inversion_filt[:100]
all_models_filt = inversion_parameters[inversion_filt, :]
all_results_filt = inversion_results[inversion_filt, :]

means = np.zeros(len(all_models_filt[1,:]))
sds = np.zeros(len(all_models_filt[1,:]))
means_grav = np.zeros(len(all_results_filt[1,:]))
sds_grav = np.zeros(len(all_results_filt[1,:]))
for i in range(len(all_models_filt[1,:])):
    means[i], sds[i] = normal_dist(all_models_filt[:, i])
for i in range(len(all_results_filt[1,:])):
    means_grav[i], sds_grav[i] = normal_dist(all_results_filt[:, i])

final_model = model(means)


In [27]:
for j in range(len(all_models_filt)):
    model_xr = surf_inside.copy()
    means_norm = all_models_filt[j]/dist_points['thick_norm']
    for i in range(len(dist_points)):
        dist_band = dist_points['distance'].iloc[i]
        model_xr = model_xr.where(dist_xr != dist_band, thick_normal_xr*means_norm[i])
    #final edit to make sure areas outside polygon are set to 0 thickness
    model_xr = model_xr.where(mask_geom_small == True, 0)
    model_xr = model_xr.where(mask_rock_small == False, 0)

    prisms_glacier = hm.prism_layer(
        (model_xr.x.values, model_xr.y.values),
        surface=surf_inside - model_xr,
        reference=surf_inside,
        properties={"density": density_model},
    )
    
    profile4_g = prisms_glacier.prism_layer.gravity((profile4['x_8N'], profile4['y_8N'], profile4['Elev']), field="g_z")
    profile7a_g = prisms_glacier.prism_layer.gravity((profile7a['x_8N'], profile7a['y_8N'], profile7a['Elev']), field="g_z")
    profile4_anom = (profile4_g - profile4_g[0]) + (profile4['outer_grav'] -  profile4['outer_grav'].iloc[0])
    profile7a_anom = (profile7a_g - profile4_g[0]) + (profile7a['outer_grav'] -  profile4['outer_grav'].iloc[0])

    thick_spline = RectBivariateSpline(model_xr.y, model_xr.x, model_xr.values)

    profile4_thick = np.empty_like(profile4_anom)
    for k in range(len(profile4_thick)):
        profile4_thick[k] = thick_spline(profile4.loc[k, 'y_8N'],profile4.loc[k,'x_8N'])[0][0]
    profile7a_thick = np.empty_like(profile7a_anom)
    for k in range(len(profile7a_thick)):
        profile7a_thick[k] = thick_spline(profile7a.loc[k, 'y_8N'],profile7a.loc[k,'x_8N'])[0][0]
    longa_thick = np.empty_like(longa.dist.to_numpy())
    for k in range(len(longa_thick)):
        longa_thick[k] = thick_spline(longa.loc[k, 'y_8N'],longa.loc[k,'x_8N'])[0][0]

    if j==0:
        p4_all_thick = profile4_thick
        p7a_all_thick = profile7a_thick
        longa_all_thick = longa_thick
        p4_all_grav = profile4_anom
        p7a_all_grav = profile7a_anom
    else:
        p4_all_thick = np.vstack((p4_all_thick, profile4_thick))
        p7a_all_thick = np.vstack((p7a_all_thick, profile7a_thick))
        longa_all_thick = np.vstack((longa_all_thick, longa_thick))
        p4_all_grav = np.vstack((p4_all_grav, profile4_anom))
        p7a_all_grav = np.vstack((p7a_all_grav, profile7a_anom))



In [28]:
means_p4_g = np.zeros(len(p4_all_grav[1,:]))
sds_p4_g = np.zeros(len(p4_all_grav[1,:]))
means_p4_t = np.zeros(len(p4_all_thick[1,:]))
sds_p4_t = np.zeros(len(p4_all_thick[1,:]))
for i in range(len(p4_all_grav[1,:])):
    means_p4_g[i], sds_p4_g[i] = normal_dist(p4_all_grav[:, i])
    means_p4_t[i], sds_p4_t[i] = normal_dist(p4_all_thick[:, i])

means_p7a_g = np.zeros(len(p7a_all_grav[1,:]))
sds_p7a_g = np.zeros(len(p7a_all_grav[1,:]))
means_p7a_t = np.zeros(len(p7a_all_thick[1,:]))
sds_p7a_t = np.zeros(len(p7a_all_thick[1,:]))
for i in range(len(p7a_all_grav[1,:])):
    means_p7a_g[i], sds_p7a_g[i] = normal_dist(p7a_all_grav[:, i])
    means_p7a_t[i], sds_p7a_t[i] = normal_dist(p7a_all_thick[:, i])

means_la_t = np.zeros(len(longa_all_thick[1,:]))
sds_la_t = np.zeros(len(longa_all_thick[1,:]))
for i in range(len(longa_all_thick[1,:])):
    means_la_t[i], sds_la_t[i] = normal_dist(longa_all_thick[:, i])

In [29]:
params_xr = surf_inside.copy()
means_norm = means/dist_points['thick_norm']
for i in range(len(dist_points)):
    dist_band = dist_points['distance'].iloc[i]
    params_xr = params_xr.where(dist_xr != dist_band, thick_normal_xr*means_norm[i])
#final edit to make sure areas outside polygon are set to 0 thickness
params_xr = params_xr.where(mask_geom_small == True, 0)
params_xr = params_xr.where(mask_rock_small == False, 0)

In [30]:
#save results
params_xr.rio.write_crs(26908, inplace=True)

dist_points['means'] = means
dist_points['sds'] = sds
longa[['grav_mean', 'mean_model_grav', 'grav_sd', 'thick_mean', 'thick_sd']] = np.c_[means_grav, final_model, sds_grav, means_la_t, sds_la_t]
profile4[['grav_mean', 'grav_sd', 'thick_mean', 'thick_sd']] = np.c_[means_p4_g, sds_p4_g, means_p4_t, sds_p4_t]
profile7a[['grav_mean', 'grav_sd', 'thick_mean', 'thick_sd']] = np.c_[means_p7a_g, sds_p7a_g, means_p7a_t, sds_p7a_t]

params_xr.rio.to_raster('Results/wide_U_shape/final_thick_xr.tif')
dist_points.to_csv('Results/wide_U_shape/dist_points_res.csv', index=False)
longa.to_csv('Results/wide_U_shape/longa_grav_res.csv', index=False)
profile4.to_csv('Results/wide_U_shape/profile4_grav_res.csv', index=False)
profile7a.to_csv('Results/wide_U_shape/profile7a_grav_res.csv', index=False)